In [ ]:
!pip uninstall scikit-learn -y
!pip install -U scikit-learn
!pip uninstall imbalanced-learn -y
!pip install -U imbalanced-learn

Found existing installation: scikit-learn 0.24.2
Uninstalling scikit-learn-0.24.2:
  Successfully uninstalled scikit-learn-0.24.2
  Using cached scikit_learn-0.24.2-cp37-cp37m-manylinux2010_x86_64.whl (22.3 MB)


Found existing installation: imbalanced-learn 0.8.0
Uninstalling imbalanced-learn-0.8.0:
  Successfully uninstalled imbalanced-learn-0.8.0
  Using cached imbalanced_learn-0.8.0-py3-none-any.whl (206 kB)


In [ ]:
import sklearn
sklearn.__version__

'0.24.2'

In [ ]:
import imblearn
imblearn.__version__

'0.8.0'

In [ ]:
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction import text 
from sklearn.ensemble import VotingClassifier 
from sklearn.linear_model import LogisticRegression 
from sklearn.svm import SVC
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import f1_score

from imblearn.over_sampling import RandomOverSampler

import random
random.seed(0)

Import training set

In [ ]:
from google.colab import files
uploaded = files.upload()
# Local Machine Path :  One_drive\BDBA\Analytics 4\BERT_Flask_Azure_Deep_Learning\Datasets\combined_dataset\combined_dataset.csv

Saving combined_dataset.csv to combined_dataset.csv


In [ ]:
import io
df = pd.read_csv( io.BytesIO( uploaded['combined_dataset.csv'] ) )
df

,text,binary_classes,multiple_classes
0,#DhoniKeepsTheGlove | WATCH: Sports Minister K...,NOT,NONE
1,@politico No. We should remember very clearly ...,cyberbullying,HATE
2,@cricketworldcup Guess who would be the winner...,NOT,NONE
3,Corbyn is too politically intellectual for #Bo...,NOT,NONE
4,All the best to #TeamIndia for another swimmin...,NOT,NONE
...,...,...,...
13480,Brokeback Mountain was boring.,cyberbullying,NaN
13481,So Brokeback Mountain was really depressing.,cyberbullying,NaN
13482,"As I sit here, watching the MTV Movie Awards, ...",cyberbullying,NaN
13483,Ok brokeback mountain is such a horrible movie.,cyberbullying,NaN


In [ ]:
category_to_binary = {'cyberbullying': 0, 'NOT': 1}
df['binary_classes'].replace(category_to_binary, inplace=True)

Information about the output classes

1 = positive sentiment, 0 = negative sentiment

In [ ]:
X = df[['text']]
y = df[['binary_classes']]

In [ ]:
print(type(X))
print(type(y))

<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>


In [ ]:
# Split data in train and test with 0.2 factor
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [ ]:
print(type(X_train))
print(type(y_train))
print(type(X_test))
print(type(y_test))

<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>


In [ ]:
print(y_train.shape) 
print(y_train.value_counts())  # y_train unbalanced: need to be under/oversampled

(10788, 1)
binary_classes
1                 6256
0                 4532
dtype: int64


In [ ]:
# Oversample the data
print(X_train.shape)

ros = RandomOverSampler(random_state=0)
X_train_oversampled, y_train_oversampled = ros.fit_resample(X_train, y_train)

print(X_train_oversampled.shape)

(10788, 1)
(12512, 1)


In [ ]:
print(type(X_train_oversampled))
print(type(y_train_oversampled))

<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>


In [ ]:
df_train = pd.concat([X_train_oversampled, y_train_oversampled], axis = 1)
df_train

,text,binary_classes
0,Again Congress started abusing PM from day 1. ...,0
1,"@hirobotix Of course, at the scent of his mast...",1
2,"@GOP @realDonaldTrump He’s also lied, cheated,...",0
3,"@VioletIrwin76 Humanity lasted this long, why ...",0
4,"EVMs were first used in 1982. In 1988, Parliam...",1
...,...,...
12507,RT @spiderlingdaya: LMAOOO ZENDAYA SAID “ill o...,0
12508,RT @KylieJenner: KYLIE F*CKING SKIN! wow. \r\n...,0
12509,"God, anyway for anyone whos read this far the ...",0
12510,How to fuck your footie career before its even...,0


In [ ]:
print(df_train['binary_classes'].value_counts(dropna=False)) #data balanced after oversampling

1    6256
0    6256
Name: binary_classes, dtype: int64


In [ ]:
X_train = df_train.iloc[:, 0]  # get all rows of the first column
y_train = df_train.iloc[:, 1]  # get all rows of the second column

In [ ]:
print(type(X_train))
print(X_train.shape)

print(type(y_train))
print(y_train.shape)

print(type(X_test))
print(X_test.shape)

print(type(y_test))
print(y_test.shape)

<class 'pandas.core.series.Series'>
(12512,)
<class 'pandas.core.series.Series'>
(12512,)
<class 'pandas.core.frame.DataFrame'>
(2697, 1)
<class 'pandas.core.frame.DataFrame'>
(2697, 1)


In [ ]:
# convert to pandas series
X_test = X_test.squeeze()
y_test = y_test.squeeze()

In [ ]:
print(type(X_train))
print(X_train.shape)

print(type(y_train))
print(y_train.shape)

print(type(X_test))
print(X_test.shape)

print(type(y_test))
print(y_test.shape)

<class 'pandas.core.series.Series'>
(12512,)
<class 'pandas.core.series.Series'>
(12512,)
<class 'pandas.core.series.Series'>
(2697,)
<class 'pandas.core.series.Series'>
(2697,)


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer as countvec

# Create customized stopping words set by including english stop words and @USER and {{URL}}
my_additional_stop_words = {'@USER', '{{URL}}', 'url', 'user'}
stop_words = text.ENGLISH_STOP_WORDS.union(my_additional_stop_words)

# Bag of Words and delete stopping words
countvec = countvec(strip_accents='unicode', stop_words=stop_words, analyzer='word')

X_train_counts = countvec.fit_transform(X_train)
X_test_counts = countvec.transform(X_test)

# Why fit_transform() on train data and transform() on test data?
# https://towardsdatascience.com/what-and-why-behind-fit-transform-vs-transform-in-scikit-learn-78f915cf96fe

# Initialize classifiers
log_clf = LogisticRegression(n_jobs=-1)
sgd_clf = SGDClassifier(loss='modified_huber', n_jobs=-1)
svm_clf = SVC(probability=True)
mnb_clf = MultinomialNB()
voting_clf = VotingClassifier( 
    estimators=[('lr', log_clf), ('svm', svm_clf), ('mnb', mnb_clf)], 
    voting='soft', n_jobs=-1)

#Train and evaluation
for clf in (log_clf, svm_clf, mnb_clf, voting_clf):
    clf.fit( X_train_counts, y_train )
    y_pred = clf.predict(X_test_counts)
    print(clf.__class__.__name__, f1_score(y_test, y_pred, average = 'macro')) 

LogisticRegression 0.7774494137250483
SVC 0.7791690204159132
MultinomialNB 0.7148849401810784
VotingClassifier 0.7848905890192499


##### CountVectorizer + LogisticRegression using Pipeline

In [ ]:
from sklearn.pipeline import Pipeline

countvec_loglr_pipe = Pipeline( [ ('vect',countvec), ('lr',log_clf) ] )

# Internally CountVectorizer.fit_transform() will be applied to X_train 
# and then X_train will be passed to LogisticRegression
countvec_loglr_pipe.fit(X_train, y_train)

# Internally CountVectorizer.transform() will be applied to X_test 
# and then X_test will be passed to LogisticRegression.predict()
y_pred = countvec_loglr_pipe.predict(X_test)

print(f"Logistic Regression F1 Score: {f1_score(y_test, y_pred, average = 'macro')}")

Logistic Regression F1 Score: 0.7774494137250483


In [ ]:
# current working directory
!pwd

/content


Saving CountVectorizer+LogisticRegression model as Pickle file

In [ ]:
import pickle

model_file_name = "countvec_loglr.pkl"

with open(model_file_name, 'wb') as picklefile:
    pickle.dump(countvec_loglr_pipe, picklefile)

Load CountVectorizer+LogisticRegression model from Pickle file

In [ ]:
with open(model_file_name, 'rb') as trained_model:
    countvec_loglr_model = pickle.load(trained_model)

text = "You are bad sexist person"

y_pred = countvec_loglr_model.predict([text])

print(y_pred)

[1]


##### CountVectorizer + SVM using Pipeline

In [ ]:
countvec_SVM_pipe = Pipeline( [ ('vect',countvec), ('svm',svm_clf) ] )

# Internally CountVectorizer.fit_transform() will be applied to X_train 
# and then X_train will be passed to SVM
countvec_SVM_pipe.fit(X_train, y_train)

# Internally CountVectorizer.transform() will be applied to X_test 
# and then X_test will be passed to SVM.predict()
y_pred = countvec_SVM_pipe.predict(X_test)

print(f"SVM F1 Score: {f1_score(y_test, y_pred, average = 'macro')}")

# Saving CountVectorizer+SVM model as Pickle file
model_file_name = "countvec_SVM.pkl"

with open(model_file_name, 'wb') as picklefile:
    pickle.dump(countvec_SVM_pipe, picklefile)

SVM F1 Score: 0.7791690204159132


Load CountVectorizer + SVM model from Pickle file

In [ ]:
with open(model_file_name, 'rb') as trained_model:
    countvec_SVM_model = pickle.load(trained_model)

text = "You are bad sexist person"

y_pred = countvec_SVM_model.predict([text])

print(y_pred)

[1]


##### CountVectorizer+MultinomialNB using Pipeline

In [ ]:
countvec_MultinomialNB_pipe = Pipeline( [ ('vect',countvec), ('multiNB',mnb_clf) ] )

# Internally CountVectorizer.fit_transform() will be applied to X_train 
# and then X_train will be passed to MultinomialNB
countvec_MultinomialNB_pipe.fit(X_train, y_train)

# Internally CountVectorizer.transform() will be applied to X_test 
# and then X_test will be passed to MultinomialNB.predict()
y_pred = countvec_MultinomialNB_pipe.predict(X_test)

print(f"MultinomialNB F1 Score: {f1_score(y_test, y_pred, average = 'macro')}")

# Saving CountVectorizer+MultinomialNB model as Pickle file
model_file_name = "countvec_multiNB.pkl"

with open(model_file_name, 'wb') as picklefile:
    pickle.dump(countvec_MultinomialNB_pipe, picklefile)

MultinomialNB F1 Score: 0.7148849401810784


Load CountVectorizer+MultinomialNB model from Pickle file

In [ ]:
with open(model_file_name, 'rb') as trained_model:
    countvec_multiNB_model = pickle.load(trained_model)

text = "You are bad sexist person"

y_pred = countvec_multiNB_model.predict([text])

print(y_pred)

[0]


##### CountVectorizer+VotingClassifier using Pipeline

In [ ]:
voting_clf = VotingClassifier( estimators=[('lr', log_clf), ('svm', svm_clf), ('mnb', mnb_clf)], voting='soft', n_jobs=1)  # gives error when n_jobs=-1
# while in the above definition does not give error

countvec_voteCLF_pipe = Pipeline( [ ('vect',countvec), ('voteCLF',voting_clf) ] )

# Internally CountVectorizer.fit_transform() will be applied to X_train 
# and then X_train will be passed to VotingClassifier
countvec_voteCLF_pipe.fit(X_train, y_train)

# Internally CountVectorizer.transform() will be applied to X_test 
# and then X_test will be passed to VotingClassifier.predict()
y_pred = countvec_voteCLF_pipe.predict(X_test)

print(f"Voting Classifier F1 Score: {f1_score(y_test, y_pred, average = 'macro')}")

# Saving CountVectorizer+VotingClassifier model as Pickle file
model_file_name = "countvec_voteCLF.pkl"

with open(model_file_name, 'wb') as picklefile:
    pickle.dump(countvec_voteCLF_pipe, picklefile)

Voting Classifier F1 Score: 0.7848905890192499


Load CountVectorizer+VotingClassifier model from Pickle file

In [ ]:
with open(model_file_name, 'rb') as trained_model:
    countvec_voteCLF_model = pickle.load(trained_model)

text = "Ok brokeback mountain is such a horrible movie"

y_pred = countvec_voteCLF_model.predict([text])

print(y_pred)

[0]
